# The University of Melbourne, School of Computing and Information Systems
# COMP90049 Introduction to Machine Learning, 2020 Semester 1
-----
## Project 1: Understanding Student Success with Naive Bayes
-----
###### Student Name(s): James Barnes, 820946
###### Python version: 3.7.6
###### Submission deadline: 11am, Wed 22 Apr 2019

This iPython notebook is a template which you will use for your Project 1 submission. 

Marking will be applied on the five functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

In [1]:
from sys import version as py_version
from collections import Counter, defaultdict as dd
from math import log, inf

import pandas as pd
import numpy as np


print(f"Python version: {py_version}")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version:  {np.__version__}")


Python version: 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]
Pandas version: 1.0.2
Numpy version:  1.18.1


In [2]:
# This function should open a data file in csv, and transform it into a usable format 
def load_data(filename):
    return pd.read_csv(filename)


In [3]:
# This function should split a data set into a training set and hold-out test set
def split_data(data, class_label, frac=0.5, **kwargs):
    tr = data.sample(frac=frac, **kwargs).sort_index()
    te = data[~data.index.isin(tr.index)]
    
    tr_labels = tr[class_label]
    te_labels = te[class_label]
    
    tr = tr.drop(columns=class_label)
    te = te.drop(columns=class_label)
    
    return tr, te, tr_labels, te_labels


In [4]:
# I have chosen to implement the NB classifier as a class to keep the namespace clean 
# for implementing the One R Classifier
class NaiveBayesClassifier:
    def train(self, data, labels, a=1):
        n = data.shape[0]
        n_labels = Counter(labels.tolist())
        n_attrs = { attr: data[attr].nunique() for attr in data.columns }
        denoms = { attr: { label: a * n_attrs[attr] + n_labels[label] 
                  for label in n_labels } for attr in data.columns } 
        
        priors = { label: count / n for label, count in n_labels.items() }
        likelihoods = dd(dict)
        for attr in data.columns:
            for label in n_labels:
                likelihoods[attr][label] = dd(lambda: a / denoms[attr][label])
                for val, count in Counter(data[labels == label][attr].tolist()).items():
                    likelihoods[attr][label][val] = (a + count) / denoms[attr][label]
                    
        self.priors = priors
        self.likelihoods = likelihoods
        
        return self
    
    def predict(self, instances):
        predictions = pd.Series(dtype='object')
        
        for i in instances.index:
            instance = list(instances.loc[i].items())
            best_label, max_prob = None, -inf
            for label, prior in self.priors.items():
                prob = log(prior) \
                       + sum(log(self.likelihoods[attr][label][val]) for attr, val in instance)
                if prob > max_prob:
                    best_label = label
                    max_prob = prob
            predictions.loc[i] = best_label 
            
        return predictions


In [5]:
def evaluate(actual_labels, predicted_labels):
    return sum(actual_labels == predicted_labels) / actual_labels.shape[0]


In [6]:
labels = ["A+", "A", "B", "C", "D", "F"]
data = load_data("./student.csv")

tr, te, tr_labels, te_labels = split_data(data, "Grade", frac=0.5, random_state=3)

model = NaiveBayesClassifier().train(tr, tr_labels, a=0.01)
predictions = model.predict(te)

accuracy = evaluate(te_labels, predictions)
print(f"accuracy: {accuracy:.4f}")

# # uncomment to see some accurate and inaccurate instances
# with pd.option_context('display.max_columns', None):
#     print("accurate")
#     print(data.loc[predictions[predictions == te_labels].index].head(15))

#     print("inaccurate")
#     print(data.loc[predictions[predictions != te_labels].index].head(15))


accuracy: 0.3600
accurate
   school sex address famsize Pstatus  Medu  Fedu      Mjob      Fjob  reason  \
4      MS   F       R     GT3       T   mid   mid     other     other   other   
5      MS   M       U     LE3       T  high  high   teacher    health   other   
6      GP   F       R     GT3       T   low   low   at_home     other    home   
7      GP   M       U     LE3       T   mid   mid  services    health    home   
9      GP   M       U     GT3       T   mid   mid  services   at_home    home   
17     MS   M       R     GT3       T  high  high    health     other  course   
21     GP   M       U     LE3       T   mid   low   at_home     other  course   
33     MS   F       R     GT3       T  high  high     other   teacher   other   
64     GP   M       U     LE3       T   mid   low  services     other  course   
65     GP   F       R     GT3       T   mid   mid     other     other    home   
72     GP   F       U     GT3       T   mid   mid     other     other    home   
84

### QUESTION 1

**a)**

The naive assumption made in Naive Bayes is that the event of seeing any given attribute is indendent of all other given attributes. 
 
This assumption is necessary as it allows us to simplify the expression containing chained conditional probabilities into a product of simpler probabilities, leading to a simpler model. 
 
This assumption is unrealistic, as there may be some underlying dependence of attributes, however finding such interdependence is likely too difficult to accurately model generally. This interdependence of variables could be seen with the traveltime and address attributes, where an urban student would be more likely to have a shorter travel time, assuming their school is urban; however, calculating the conditional probability of such events occuring does not seem feasible.
 
 
**b)**

(Implementation above)
 
 
**c)**

The model's accuracy score sits around 35%, depending on the exact split used, as well as the alpha value used in the Laplacian smoothing. This typically varies by around 2%.
 
I was unable to find any strong patterns in both the accurate and inaccurate predictions the model made that were not already present in the other. With a data set with so many features, it can be very hard for a human to analyze and interpret, as there is many variables. This is a primary reason for utilising machine learned models, as they have the ability to learn complex patterns in complex datasets, such as this one.
 
 One such example of a pattern that I attempted to find was a relationship between the classes that are correctly predicted when the value of 'Pstatus' is 'A'. In the accurate, these are 'D' and 'C' in the instances I looked at, while in the innacturate these were 'D' and 'F'. Or for the 'internet' feature, where instances have the value 'no', the accurately predicted instances are 'C' and 'D', while the innacurate are also 'C' and 'D'. I do not think that either of these pattern hold any weight in understanding how the classifier makes its predictions.

In [7]:
# question 2
def precision(vals):
    tp, _, fp, _ = vals
    return tp / (tp + fp) if tp + fp else 0

def recall(vals):
    tp, _, _, fn = vals
    return tp / (tp + fn) if tp + fn else 0

def f1(p, r):
    return 2 * p * r / (p + r) if p + r else 0

def confusion(actual_labels, predicted_labels, interesting):
    actual_labels = actual_labels == interesting
    predicted_labels = predicted_labels == interesting
    return sum(actual_labels  &  predicted_labels), \
           sum(~actual_labels & ~predicted_labels), \
           sum(~actual_labels &  predicted_labels), \
           sum(actual_labels  & ~predicted_labels)

def further_evaluate(actual_labels, predicted_labels, labels):
    n = len(labels)
    p_m, r_m, label_values = 0, 0, []
    for interesting in labels:
        vals = confusion(actual_labels, predicted_labels, interesting)
        p = precision(vals)
        r = recall(vals)
        label_values.append((interesting, (p, r, f1(p, r))))
        p_m += p; r_m += r
    p_m /= n; r_m /= n
    label_values.append(("macro", (p_m, r_m, f1(p_m, r_m))))
    return label_values

def pprint_evals(*evals):
    for label, (p, r, f) in evals:
        print(f"{label:>5s}: prec = {p:.5f}, rec = {r:.5f}, f1 = {f:.5f}")


In [8]:
# question 2
evals = further_evaluate(te_labels, predictions, labels)
pprint_evals(*evals)


   A+: prec = 0.33333, rec = 0.12500, f1 = 0.18182
    A: prec = 0.23529, rec = 0.22857, f1 = 0.23188
    B: prec = 0.24490, rec = 0.20339, f1 = 0.22222
    C: prec = 0.37363, rec = 0.43590, f1 = 0.40237
    D: prec = 0.45161, rec = 0.42000, f1 = 0.43523
    F: prec = 0.36364, rec = 0.44444, f1 = 0.40000
macro: prec = 0.33373, rec = 0.30955, f1 = 0.32119


### QUESTION 2
**a)**

Accuracy is a measure of how often our model is correct at predicting an instance's class. A high accuracy measure corresponds with a low FP and FN count, which means that the model gets a lot of predictions correct.
 
Precision is a measure of how good a model is at predicting an instance to be interesting, when it is actually interesting. A high precision measure corresponds with a low FP count, meaning the model often does not predict that an instance should be interesting if it is not.
 
Recall is a measure of how good our model is at not missing interesting instances. A high recall corresponds to a low FN count, meaning the model often does not predict an interesting instance as uninteresting.
 
F-1 is the harmonic mean of recall and precision, giving a different repesentation to the overall correctness of the predictions our model makes. A high F-1 score corresponds with a low FP and FN count, which means that our model generally gets more predictions correct than not.
 
Macro-averaging averages the recall and or precision over all classes, treating each class with the same weight. Micro-averaging takes class inbalances into account better, providing a more representative average based on the size of each class. 
 
For instance, the 'A' and 'A+' classes have relatively small sizes. As the 'A' and 'A+' classes have drastically different recall and precision metrics to the other classes, the macro-averages will dispropotionately contribute more weight to these classes, whereas the micro-average will provide a better, proportionally weighted, average of the measures.
 
**b)**

(Implementation above)
 
All scores, precision, recall, and F-1 are typically below that of the accuracy. These values generally average around 30%.
 
Generally, I have found, the model performs better (higher precision, recall, and F-1 scores) when the class size is larger. However, this tread is not monotonic. The 'F' class often attains the highest scores, however it is not the largest class (the 'C' class is). I believe that these classes must be more closely associated with features that have higher probabilities, as this would mean they are more frequently predicted, and generally more correctly. 


In [9]:
# question 4
class OneRClassifier:
    def train(self, data, labels):
        default_label = labels.mode()[0]
        
        best_rule, min_err = None, inf
        for attr in data.columns:
            rule = [(val, labels[data[attr] == val].mode()[0]) 
                              for val in data[attr].unique()]
            err = sum(sum(labels[data[attr] == val] != label) for val, label in rule)
            if err < min_err:
                best_rule = (attr, rule)
                min_err = err
            
        self.attr, rule = best_rule
        self.rule_dict = dd(lambda: default_label, rule)
        
        return self
        
    def predict(self, instances):
        predictions = pd.Series(index=instances.index, dtype='object')
        for i in predictions.index:
            predictions.loc[i] = self.rule_dict[instances.loc[i, self.attr]]
            
        return predictions

# class ZeroRClassifier:
#     def train(self, data, labels):
#         self.label = labels.mode()[0]
        
#         return self
        
#     def predict(self, instances):
#         predictions = pd.Series(index=instances.index, dtype='object', 
#                                 data=[self.label] * instances.shape[0])

#         return predictions


In [10]:
# question 4
model = OneRClassifier().train(tr, tr_labels)
predictions = model.predict(te)

accuracy = evaluate(te_labels, predictions)
print(f"accuracy: {accuracy:.4f}")

evals = further_evaluate(te_labels, predictions, labels)
pprint_evals(*evals) 

accuracy: 0.2831
   A+: prec = 0.00000, rec = 0.00000, f1 = 0.00000
    A: prec = 0.00000, rec = 0.00000, f1 = 0.00000
    B: prec = 0.20000, rec = 0.08475, f1 = 0.11905
    C: prec = 0.25455, rec = 0.53846, f1 = 0.34568
    D: prec = 0.33333, rec = 0.45000, f1 = 0.38298
    F: prec = 0.00000, rec = 0.00000, f1 = 0.00000
macro: prec = 0.13131, rec = 0.17887, f1 = 0.15145


### QUESTION 4
**a)** 
 
I have chosen to use the One-R classification method.
 
The One-R method is a simple decision-based classification method, effictively being a decision tree with a single decision node. In the training phase, the model chooses a single, simple rule to then use in its predictions. 

The rule is chosen as follows:

    for each attribute:
      for each value of the attribute:
        rule: assign the most frequent class associated with the instances which 
            have this value for the attribute
        calculate the error of assigning these instances with this 
            class, uning this rule
    choose the rule which minimises the total error over each value
    
The prediction step is then as follows:

    for each instance:
      assign it the class which the rule assigns this instance's value for the 
          selected attribute
    
I chose this model for its general simplicity, as well as for it's use as a standard baseline in ML contexts
 
**b)** 
 
(Implementation above)
 
In my testing, I have found that the Naive Bayes Classifier outperformed the One R Classifier by approximately 5% in terms of accuracy, and generally far better in terms of (macro) recall, precision, and f-1 scores.
 
The One-R classifier often classifies all instances as either 'D' or 'C', and rarely as 'B' and 'F', likely because there are more instances with these labels. As the 'D' and 'C' classes account for 54% of all instances, the classifier can never achieve an accuracy score higher than 54%. The One-R classifier also often choses to make it's rule based on the 'Fedu' feature. This is also why the macro scores are much bettwer with the Naive Bayes classifier, as many of the classes score 0.
 
The increased performance of the NB classifier is likely due to it being able to predict some of the other 46% of instance's classes, while not predicting the 'C' and 'D' instances as well. 

## Questions (you may respond in a cell or cells below):

You should respond to Question 1 and two additional questions of your choice. A response to a question should take about 100–250 words, and make reference to the data wherever possible.

### Question 1: Naive Bayes Concepts and Implementation

- a Explain the ‘naive’ assumption underlying Naive Bayes. (1) Why is it necessary? (2) Why can it be problematic? Link your discussion to the features of the students data set. [no programming required]
- b Implement the required functions to load the student dataset, and estimate a Naive Bayes model. Evaluate the resulting classifier using the hold-out strategy, and measure its performance using accuracy.
- c What accuracy does your classifier achieve? Manually inspect a few instances for which your classifier made correct predictions, and some for which it predicted incorrectly, and discuss any patterns you can find.

### Question 2: A Closer Look at Evaluation

- a You learnt in the lectures that precision, recall and f-1 measure can provide a more holistic and realistic picture of the classifier performance. (i) Explain the intuition behind accuracy, precision, recall, and F1-measure, (ii) contrast their utility, and (iii) discuss the difference between micro and macro averaging in the context of the data set. [no programming required]
- b Compute precision, recall and f-1 measure of your model’s predictions on the test data set (1) separately for each class, and (2) as a single number using macro-averaging. Compare the results against your accuracy scores from Question 1. In the context of the student dataset, and your response to question 2a analyze the additional knowledge you gained about your classifier performance.

### Question 3: Training Strategies 

There are other evaluation strategies, which tend to be preferred over the hold-out strategy you implemented in Question 1.
- a Select one such strategy, (i) describe how it works, and (ii) explain why it is preferable over hold-out evaluation. [no programming required]
- b Implement your chosen strategy from Question 3a, and report the accuracy score(s) of your classifier under this strategy. Compare your outcomes against your accuracy score in Question 1, and explain your observations in the context of your response to question 3a.

### Question 4: Model Comparison

In order to understand whether a machine learning model is performing satisfactorily we typically compare its performance against alternative models. 
- a Choose one (simple) comparison model, explain (i) the workings of your chosen model, and (ii) why you chose this particular model. 
- b Implement your model of choice. How does the performance of the Naive Bayes classifier compare against your additional model? Explain your observations.

### Question 5: Bias and Fairness in Student Success Prediction

As machine learning practitioners, we should be aware of possible ethical considerations around the
applications we develop. The classifier you developed in this assignment could for example be used
to classify college applicants into admitted vs not-admitted – depending on their predicted
grade.
- a Discuss ethical problems which might arise in this application and lead to unfair treatment of the applicants. Link your discussion to the set of features provided in the students data set. [no programming required]
- b Select ethically problematic features from the data set and remove them from the data set. Use your own judgment (there is no right or wrong), and document your decisions. Train your Naive Bayes classifier on the resulting data set containing only ‘unproblematic’ features. How does the performance change in comparison to the full classifier?
- c The approach to fairness we have adopted is called “fairness through unawareness” – we simply deleted any questionable features from our data. Removing all problematic features does not guarantee a fair classifier. Can you think of reasons why removing problematic features is not enough? [no programming required]
